<a href="https://colab.research.google.com/github/JVL01/BITS_Dissertation/blob/main/Dissertation_CodeGeneration_5524_PostMid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Dissertation : Applications of Prompt Engineering Technniques in Code Generation**

### **J.Vijayalakshmi (2022AC05524)**




In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installing the libraries

In [ ]:
!pip install datasets

!pip install datasets faiss-cpu
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=99448adb9283706880f01db946e7fe67491e3ffc6a6eabc99b96004d0b19ef02
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


### Loading the dataset
### These datasets used due to their accessibilit and understanding their relvance after the literature review

In [ ]:
from datasets import load_dataset

#  Python and Java datasets from CodeXGLUE upload - Source GITHUB repo (mentioned in the literature)
python_ds = load_dataset("code_x_glue_ct_code_to_text", "python")
java_ds = load_dataset("code_x_glue_ct_code_to_text", "java")
# uploading the dataset from codeparrot - Source Hugging Face
codeparrot_ds = load_dataset("codeparrot/github-jupyter-text-code-pairs")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/10955 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/782 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

(…)-00000-of-00001-81d7ff8514320810.parquet:   0%|          | 0.00/181M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/451662 [00:00<?, ? examples/s]

### Printing the dataset of first OR/AND 5 examples
### Printing the details
### Dataset contains training and validation set.
### Tried  to print 5 rows.


In [ ]:
print(python_ds['train'][0])
print(java_ds['train'][0])
print(codeparrot_ds["train"][0])

#printing the dataset details
print(python_ds)
print(java_ds)
print(codeparrot_ds)

'''
#print(ds["train"][:5])


train_data = ds["train"]

# Display the first 5 examples with both markdown and code
for i in range(5):
    print(f"Markdown: {train_data[i]['markdown']}")
    print(f"Code: {train_data[i]['code']}")
    print()
'''


{'id': 0, 'repo': 'proycon/pynlpl', 'path': 'pynlpl/formats/folia.py', 'func_name': 'AbstractElement.settext', 'original_string': 'def settext(self, text, cls=\'current\'):\n        """Set the text for this element.\n\n        Arguments:\n            text (str): The text\n            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.\n        """\n        self.replace(TextContent, value=text, cls=cls)', 'language': 'python', 'code': 'def settext(self, text, cls=\'current\'):\n        """Set the text for this element.\n\n        Arguments:\n            text (str): The text\n            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.\n        """\n        self.replace(TextContent, value=text, cls=cls)', 'c

'\n#print(ds["train"][:5])\n\n\ntrain_data = ds["train"]\n\n# Display the first 5 examples with both markdown and code\nfor i in range(5):\n    print(f"Markdown: {train_data[i][\'markdown\']}")\n    print(f"Code: {train_data[i][\'code\']}")\n    print()\n'

### Switched to codeparrot-small variant model.Intially distilGPT was used,
### But GPT-2 was originally is a trained model for text generation.

### Better alternative is  Using AutoModelForCausalLM, which is designed for causal language modeling (used in code generation tasks).
### Using AutoModelForCausalLM instead of GPT2LMHeadModel → Optimized for code generation.
### Using AutoTokenizer instead of GPT2Tokenizer → Ensures the tokenizer matches the CodeParrot model.
### Setting pad_token_id to eos_token_id → Required for handling padding in GPT-like models.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load CodeParrot-Small model and tokenizer which is  optimized for code generation
model_name = "codeparrot/codeparrot-small"   #distilGPT

# AutoTokenizer class being used  to ensure correct tokenization for the model
# previously usee
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token for compatibility as GPT models usually do not have padding.
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id




The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/903 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/457M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/277k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

###  Preprocess the Dataset (Tokenization)
###  Before fine-tuning CodeParrot-small, the datasets needs to be tokenized
###  The three datasets are tokenized seperately
###  This dataset is then be fed into the pre trained model for model fine tuning

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples['code'], truncation=True, padding="max_length", max_length=256)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

# Tokenizint the dataset as required as input for the pre-trained model
# Tokenizing the datasets seperately

#Tokenizing Codeparrot dataset
tokenized_ds = codeparrot_ds.map(preprocess_function, batched=True)

# Tokenizing Python dataset
tokenized_python_ds = python_ds.map(preprocess_function, batched=True)

# Tokenizing Java dataset
tokenized_java_ds = java_ds.map(preprocess_function, batched=True)




Map:   0%|          | 0/451662 [00:00<?, ? examples/s]

Map:   0%|          | 0/251820 [00:00<?, ? examples/s]

Map:   0%|          | 0/13914 [00:00<?, ? examples/s]

Map:   0%|          | 0/14918 [00:00<?, ? examples/s]

Map:   0%|          | 0/164923 [00:00<?, ? examples/s]

Map:   0%|          | 0/5183 [00:00<?, ? examples/s]

Map:   0%|          | 0/10955 [00:00<?, ? examples/s]

### Storing the datasets in parquet format which is sutied to store huge data

In [ ]:
# Save each dataset separately to Google Drive in Parquet format
tokenized_ds['train'].to_parquet('/content/drive/My Drive/tokenized_train.parquet')
tokenized_python_ds['train'].to_parquet('/content/drive/My Drive/tokenized_train_python_dsparquet') # The 'test' split does not exist in this dataset
tokenized_java_ds['train'].to_parquet('/content/drive/My Drive/tokenized_train_java_ds.parquet')

Creating parquet from Arrow format:   0%|          | 0/452 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/252 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/165 [00:00<?, ?ba/s]

1003396655

### 90:10 training : testing split
### Each tokenized datasets is  split separately


In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset


# Split Python dataset
train_test_split_python = tokenized_python_ds["train"].train_test_split(test_size=0.1)
train_python = train_test_split_python["train"]
test_python = train_test_split_python["test"]

# Split Java dataset
train_test_split_java = tokenized_java_ds["train"].train_test_split(test_size=0.1)
train_java = train_test_split_java["train"]
test_java = train_test_split_java["test"]

# Split codeparrot dataset
train_test_split_codeparrot= tokenized_ds["train"].train_test_split(test_size=0.1)
train_codeparrot = train_test_split_codeparrot["train"]
test_codeparrot = train_test_split_codeparrot["test"]


### not using MLM - masked language modelling as the requirement of codeparrot model.
### predicting the next token from the batch for the model.


In [ ]:

# Define the data collator
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",  # Directory to save model checkpoints
    per_device_train_batch_size=2,  # Batch size chosen 2 more than 2 execution is taking time
    gradient_accumulation_steps=8,  # To simulate larger batch size
    num_train_epochs=1,  # Number of epochs  (reduced to 1 to check for the pipeline execution)
    logging_dir="./logs",  # Directory for logs
    logging_steps=100,
    save_steps=500,

    save_total_limit=2,
    fp16=True,  # Mixed precision for faster training
    max_steps=10,  # not approriate. need to change the steps. it is not training properly.
    report_to="none"

)


### For  fine-tuning, Loading the pre-trained model first
### Three differnet models being generated on the Base-Model, therefore Each dataset gets its own fine-tuned model.

In [ ]:
from transformers import AutoModelForCausalLM

# Base pre-trained model - codeparrot-small variant
model_python = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small")
model_java = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small")
model_codeparrot = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small")


### The following code will train three models separately:

### Python Model → Fine-tuned on Python dataset
### Java Model → Fine-tuned on Java dataset
### Jupyter Model → Fine-tuned on GitHub-Jupyter dataset

### Traing the model took approximately 10 mins each.

In [ ]:
from transformers import Trainer

# Traniner defined for Python model - training data
trainer_python = Trainer(
    model=model_python,
    args=training_args,
    train_dataset=train_python,
    eval_dataset=test_python

)

# Traniner defined for Java model - training data
trainer_java = Trainer(
    model=model_java,
    args=training_args,
    train_dataset=train_java,
    eval_dataset=test_java
)

# Traniner defined for codeparrot model - training data
trainer_codeparrot = Trainer(
    model=model_codeparrot,
    args=training_args,
    train_dataset=train_codeparrot,
    eval_dataset=test_codeparrot
)

# Start training each model
trainer_python.train()
trainer_java.train()
trainer_codeparrot.train()

# Printing the outputs of three training process.
output_python = trainer_python.train()
print("Python Trainer Output:", output_python)

output_java = trainer_java.train()
print("Java Trainer Output:", output_java)

output_codeparrot = trainer_codeparrot.train()
print("CodeParrot Trainer Output:", output_codeparrot)




`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Step,Training Loss


Step,Training Loss


Step,Training Loss


Python Trainer Output: TrainOutput(global_step=10, training_loss=1.2762810707092285, metrics={'train_runtime': 709.823, 'train_samples_per_second': 0.225, 'train_steps_per_second': 0.014, 'total_flos': 20903362560000.0, 'train_loss': 1.2762810707092285, 'epoch': 0.0007059716375894599})


Step,Training Loss


Java Trainer Output: TrainOutput(global_step=10, training_loss=1.3211462020874023, metrics={'train_runtime': 703.9928, 'train_samples_per_second': 0.227, 'train_steps_per_second': 0.014, 'total_flos': 20903362560000.0, 'train_loss': 1.3211462020874023, 'epoch': 0.0010779492016438725})


Step,Training Loss


CodeParrot Trainer Output: TrainOutput(global_step=10, training_loss=1.0257977485656737, metrics={'train_runtime': 728.0077, 'train_samples_per_second': 0.22, 'train_steps_per_second': 0.014, 'total_flos': 20903362560000.0, 'train_loss': 1.0257977485656737, 'epoch': 0.0003936078091789341})


### Saving the three models

In [ ]:
# Save the Python-trained model
model_python.save_pretrained("./trained_codeparrot_python")
tokenizer.save_pretrained("./trained_codeparrot_python")

# Save the Java-trained model
model_java.save_pretrained("./trained_codeparrot_java")
tokenizer.save_pretrained("./trained_codeparrot_java")

# Save the Codeparrot dataset model
model_codeparrot.save_pretrained("./trained_codeparrot_jupyter")
tokenizer.save_pretrained("./trained_codeparrot_jupyter")


('./trained_codeparrot_jupyter/tokenizer_config.json',
 './trained_codeparrot_jupyter/special_tokens_map.json',
 './trained_codeparrot_jupyter/vocab.json',
 './trained_codeparrot_jupyter/merges.txt',
 './trained_codeparrot_jupyter/added_tokens.json',
 './trained_codeparrot_jupyter/tokenizer.json')

###  Merging  all snippets into a single dataset
### Taking the first 5000 snippets only due of time constriant
### Later this dataset is used to  implement RAG for the prompt refining


In [ ]:

codeparrot_snippets = codeparrot_ds["train"]["code"][:5000]  # More than this is taking time. also for efficiency
python_snippets = python_ds["train"]["code"][:5000]
java_snippets = java_ds["train"]["code"][:5000]


all_code_snippets = codeparrot_snippets + python_snippets + java_snippets
print(f" Merged Dataset for RAG-Prompt Refining : {len(all_code_snippets)}")


 Merged Dataset for RAG-Prompt Refining : 15000


### CodeT5 -  It’s optimized for code-related tasks and much smaller than traditional RAG models
### Initially used, however switched to FLAN



In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load CodeT5 model for prompt refinement
model_name = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(" CodeT5 model loaded for prompt refinement.")



tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

 CodeT5 model loaded for prompt refinement.


### Generating  Embeddings for the snippets of the merged dataset
### using FAISS requires numerical embeddings, so each code snippet is encoded using All-MiniLM-L6-v2 a sentence transformers pretrained model
### This model is used to generate  vector representations of sentences text in my case the code snippets which acts as the base for FAISS to search effecitinely in the vector database, for more relevant code

### Each code snippet is being represented as a numerical vector for retrieval


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Using/Loading the sentence transformer model for embedding generation
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Converting code snippets into embeddings of the all_code_snippets mergerd dataset
all_code_embeddings = np.array(embedding_model.encode(all_code_snippets, convert_to_numpy=True))

print(f" Code embeddings successfully generated : {all_code_embeddings.shape}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Code embeddings successfully generated : (15000, 384)


### Storing these embeddings in FAISS, which allows easier search for similar code snippets efficiently across the merged dataset


In [ ]:
import faiss

# Define FAISS index for fast similarity search
dimension = all_code_embeddings.shape[1]  # Dimension of embeddings
faiss_index = faiss.IndexFlatL2(dimension)  # L2 similarity search

# Add embeddings to FAISS index
faiss_index.add(all_code_embeddings)

print(" Code embeddings and Index added to FAISS")


 Code embeddings and Index added to FAISS


In [ ]:
faiss.write_index(faiss_index, "/content/drive/MyDrive/faiss_index")
print("FAISS index saved to Google Drive!")


FAISS index saved to Google Drive!


### Applying RAG to get relevant code which is
### used to frame a structured query

In [96]:
def retrieve_similar_code(query, top_k=3):
    """Retrieve top-k similar code snippets using FAISS."""

    query_embedding = np.array(embedding_model.encode([query], convert_to_numpy=True))
    # Searching FAISS index for top-k similar results
    distances, indices = faiss_index.search(query_embedding, top_k)

    #  retrieved code snippets
    retrieved_snippets = [all_code_snippets[i] for i in indices[0]]
    return retrieved_snippets

def refine_prompt_with_retrieval(user_input, top_k=3):
    """Retrieve relevant code snippets and refine the user's prompt."""
    retrieved_snippets = retrieve_similar_code(user_input, top_k)
    # Construct refined prompt
    refined_prompt = f"Rewrite this programming task based on examples:\n"
    for i, snippet in enumerate(retrieved_snippets, 1):
        refined_prompt += f"\nExample {i}:\n{snippet}\n"

    refined_prompt += f"\nNow, generate a function based on these examples: {user_input}"
    return refined_prompt
# Example Usage
user_prompt = "factorial of a number"
refined_prompt = refine_prompt_with_retrieval(user_prompt)

print("Refined Prompt with Retrieval:")
print(refined_prompt)


Refined Prompt with Retrieval:
Rewrite this programming task based on examples:

Example 1:
def factorial(N):
    """Compute the factorial of N.
    If N <= 10, use a fast lookup table; otherwise use scipy.special.factorial
    """
    if N < len(FACTORIALS):
        return FACTORIALS[N]
    else:
        from scipy import special
        return int(special.factorial(N))

Example 2:
import math

math.sqrt(1000)

factorial(10)  # This will cause an error!

math.factorial(10)  # This is ok, since the math package comes with a function called factorial.

Example 3:
public static String multiply(CharSequence self, Number factor) {
        String s = self.toString();
        int size = factor.intValue();
        if (size == 0)
            return "";
        else if (size < 0) {
            throw new IllegalArgumentException("multiply() should be called with a number of 0 or greater not: " + size);
        }
        StringBuilder answer = new StringBuilder(s);
        for (int i = 1; i < siz

In [97]:
print(refined_prompt)

Rewrite this programming task based on examples:

Example 1:
def factorial(N):
    """Compute the factorial of N.
    If N <= 10, use a fast lookup table; otherwise use scipy.special.factorial
    """
    if N < len(FACTORIALS):
        return FACTORIALS[N]
    else:
        from scipy import special
        return int(special.factorial(N))

Example 2:
import math

math.sqrt(1000)

factorial(10)  # This will cause an error!

math.factorial(10)  # This is ok, since the math package comes with a function called factorial.

Example 3:
public static String multiply(CharSequence self, Number factor) {
        String s = self.toString();
        int size = factor.intValue();
        if (size == 0)
            return "";
        else if (size < 0) {
            throw new IllegalArgumentException("multiply() should be called with a number of 0 or greater not: " + size);
        }
        StringBuilder answer = new StringBuilder(s);
        for (int i = 1; i < size; i++) {
            answer.ap

### This structured prompt is what will be fed into CodeT5 for a more readable and effective prompt.

In [ ]:
import re

def clean_refined_prompt(text):
    """Cleans unwanted non-English terms and extra spaces."""
    text = re.sub(r'#Lösung:?', '', text)  # Remove unwanted words
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces
    return text

refined_prompt = clean_refined_prompt(refined_prompt)


In [98]:
def extract_instruction(retrieved_prompt):
    """Extract only the final instruction for refinement."""
    lines = retrieved_prompt.strip().split("\n")
    for line in reversed(lines):
        if "Now, generate" in line:
            # Ensure that we only keep the last relevant sentence
            return line.replace("Now, generate a function based on these examples:", "").strip()

    return retrieved_prompt  # If no match, return full prompt

# Extracted instruction before refining
instruction_part = extract_instruction(refined_prompt)
print("🔍 Extracted Instruction Before CodeT5:", instruction_part)




🔍 Extracted Instruction Before CodeT5: factorial of a number


In [99]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load FLAN-T5
model_name = "google/flan-t5-large"   #"Salesforce/codet5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def refine_prompt(user_prompt):
    """Refines a vague programming request into a structured prompt."""

    # 🚀 Improved Input Formatting
    input_text = f"""
    ### Task: Rewrite vague programming prompts into structured function definitions.

    ### Example 1:
    - Vague: "sum numbers"
    - Refined: "Write a Python function that takes a list of numbers and returns their sum."

    ### Example 2:
    - Vague: "check prime"
    - Refined: "Write a Python function to check whether a number is prime."

    ### Now process:
    - Vague: "{user_prompt}"
    - Refined:
    """

    # Tokenize input prompt
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding=True
    )

    # Generate refined output
    outputs = model.generate(
        inputs.input_ids,
        max_length=256,
        num_return_sequences=1,
        num_beams=5,
        repetition_penalty=1.5,  # 🚀 Prevents repeated words
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


final_refined_prompt = refine_prompt(instruction_part)
print("Refined Prompt after the structured prompt is fed into  FLAN-T5 model : \n", final_refined_prompt)


Refined Prompt after the structured prompt is fed into  FLAN-T5 model : 
 "Write a Python function that returns the factorial of a number."


In [101]:
print(final_refined_prompt)

"Write a Python function that returns the factorial of a number."


In [49]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load FLAN-T5
model_name = "Salesforce/codet5-base"  #"google/flan-t5-large"   #
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def refine_prompt(user_prompt):
    """Refines a vague programming request into a structured prompt."""

    # 🚀 Improved Input Formatting
    input_text = f"""
    ### Task: Rewrite vague programming prompts into structured function definitions.

    ### Example 1:
    - Vague: "sum numbers"
    - Refined: "Write a Python function that takes a list of numbers and returns their sum."

    ### Example 2:
    - Vague: "check prime"
    - Refined: "Write a Python function to check whether a number is prime."

    ### Now process:
    - Vague: "{user_prompt}"
    - Refined:
    """

    # Tokenize input prompt
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding=True
    )

    # Generate refined output
    outputs = model.generate(
        inputs.input_ids,
        max_length=256,
        num_return_sequences=1,
        num_beams=5,
        repetition_penalty=1.5,  # 🚀 Prevents repeated words
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


final_refined_prompt_Codet5 = refine_prompt(instruction_part)
#print("Refined Prompt after the structured prompt is fed into  FLAN-T5 model : \n", final_refined_prompt)
print("Refined Prompt after the structured prompt is fed into  Codet5 model : \n", final_refined_prompt_Codet5)


Refined Prompt after the structured prompt is fed into  Codet5 model : 
 - Vague: "check prime"a Python function to checka number is prime."- Refined: "Write a Pythonfunction to check whethera number is prime."- Vague: "check prime"- Refined: "Write aPython function to checka number is prime.""Checkprime."1:
    - Vague: "check- Refined: "Write a


In [50]:
print(final_refined_prompt_Codet5)

- Vague: "check prime"a Python function to checka number is prime."- Refined: "Write a Pythonfunction to check whethera number is prime."- Vague: "check prime"- Refined: "Write aPython function to checka number is prime.""Checkprime."1:
    - Vague: "check- Refined: "Write a


In [91]:
def generate_code(prompt):

    tokenizer.pad_token = tokenizer.eos_token


    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)


    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=250, #500 , 300 changed this for more structured prompt ouput. also for TOT prompt
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.3,  #changed to from 0.3 to 0.5 for tot
        top_p=0.8,  #changed from 0.5 to 0.8 for tot
        top_k=40,  # changed from 50 to 40
        pad_token_id=tokenizer.eos_token_id,  # Ensure padding token is set
    )


    generated_code = tokenizer.decode(outputs[  0], skip_special_tokens=True)
    # Ensure proper formatting by replacing "; " and other indicators
    formatted_code = generated_code.replace("; ", ";\n").replace(":", ":\n").replace("{", "{\n").replace("}", "}\n").replace("for ", "\nfor ").replace("while ", "\nwhile ").replace("if ", "\nif ")

    return formatted_code  #formatted for the sake of printing on vertical lines and producting
                           # spaces between the code lines for readability purposes





### Testing the model with a direct question

In [102]:
 prompt_zero_shot = generate_code(final_refined_prompt)  # input the refined query. first attempt as zero shot

print(prompt_zero_shot)


n = int(input()) 
if 0  i % 2 == -1:
 print(n) else:
 x = '' 
for k in range(2, 10):
 
for (k * j) in enumerate(k):
 # print (x) # y = (y * (j + 1)) // 2 - b / c a = 1 
while len(a) > m:
 mid = mid + (l + d) * 2 + 1 s = "" t = str(t) + " " + str() r = len(s) print (""".join(map(lambda num:
 str.get(r)) 
for str in str)+" "+str(x + 2))


In [104]:
generated_code_ZS1 = generate_code(final_refined_prompt)
print(generated_code_ZS1)

n = int(input()) 
if 0  m % 2 != st :
 print(n) else:
 i = -1 
while j - k > 1:
 
while (i + l ) // 2 == 1 and a & b = = 1 / 2:
 ind = (j + (l + 1) // 2) t = 2 * inf dp = [[0] * (n + 2) 
for _ in range(1, 10)] 
for item in sorted(items):
 
for element in item:
d
if = item[-1] res = list(df[:
i]) print()


In [103]:
generated_code_ZS2 = generate_code(final_refined_prompt)
print("Zero-Shot Output:\n", generated_code_ZS2)

Zero-Shot Output:
 n = int(input()) 
if 0 = '0' != 1:
 print('1') else:
 m = float("inf") f = 1 
while s % p ==0:
 
for i in range(1, 5):
 
while (s // pp = = 1):
 #print(i) 
for item in sorted(list(filter(lambda x:
 item - 1, num))), reverse = True):
 now = item[0] res = now / tuple([i]) print()


In [62]:
prompt_FS1 = """
# Example 1
# Task: Write a Python function to print 'Hello, World!'
def print_hello_world():
    print('Hello, World!')

# Example 2
# Task: Write a Python function to calculate the square of a number.
def square_number(num):
    return num * num

# Example 3
# Task: Write a Python function to calculate the cube of a number.

# final_refined_prompt
"""


In [112]:

generated_code_FS1 = generate_code(final_refined_prompt)
print("Few-Shot Output:\n", generated_code_FS1)

Few-Shot Output:
 n = int(input()) 
if 0  st == '0':
 print('1') else:
 a = -1 
for i in range(1, 7):
 
for ab in f:
a += ab - m % b el
if ab > d:
d = ab print()


In [105]:
prompt_FS2 = """
# Example 1:
# Task: Write a Python function to add two numbers.
def add_numbers(a, b):
    return a + b

# Example 2:
# Task: Write a Python function to find the maximum of two numbers.
def max_number(a, b):
    return max(a, b)

# refined_prompt
"""

generated_code_FS2 = generate_code(final_refined_prompt)
print("Few-Shot Output:\n", generated_code_FS2)


Few-Shot Output:
 n = int(input()) 
if 0 = m % 2 == 1:
 print(n) else:
 x = -1 
while y != 2:
 
while (y > 3):
 mid = (x + 6) // 2 - 1 t = mid / 2 print (t)


In [106]:

prompt_cot = """
# Let's solve the factorial problem step by step.
# Step 1: Define the base case (factorial of 0 is 1).
# Step 2: Define the recursive formula (n * factorial(n-1)).
# Step 3: Implement the function.

Now, generate a function: {final_refined_prompt}
"""


generated_code_cot = generate_code(prompt_cot)
print("Chain-of-Thought Output:\n", generated_code_cot)


Chain-of-Thought Output:
 n = int(input()) res = 0 
while i  len(n):
 
if m == -1:
 break el
if (n % 3) = = 1:
 print("".join(map(str, ' :
')) + "".format(m));
a = (m - 4) * (l / 6) + 1;
b = 2 * 7) + (a & c) * 2;
print();


In [107]:
prompt_cot_FS = """
# Example 1:
# Task: Compute the sum of a list of numbers.
def sum_list(lst):
    total = 0
    for num in lst:
        total += num
    return total

# Example 2:
# Task: Compute the factorial of a number.
# Let's solve this step by step.
# Step 1: Define the base case (factorial of 0 is 1).
# Step 2: Define the recursive formula (n * factorial(n-1)).
# Step 3: Implement the function.


Now, generate a function: {final_refined_prompt}

"""


generated_code_cot_FS = generate_code(prompt_cot_FS)
print("Few shot trained before Chain-of-Thought Outputs :\n", generated_code_cot_FS)


Few shot trained before Chain-of-Thought Outputs :
 n = int(input()) res = 0 
for i in range(n):
 
if a == -1:
 break el
if (n % 3) = = 1:
 print(i) exit()


In [108]:
prompt_tot = """
# Let's explore multiple ways to implement the Fibonacci series.
# Approach 1: Using recursion.
def fibonacci_recursive(n):
    if n <= 1:
        return n
    return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

# Approach 2: Using iteration.
def fibonacci_iterative(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

# Approach 3: Using memoization for efficiency.
def fibonacci_memoization(n, memo={}):
    if n in memo:
        return memo[n]
    if n <= 1:
        return n
    memo[n] = fibonacci_memoization(n - 1, memo) + fibonacci_memoization(n - 2, memo)
    return memo[n]

# Now, generate another optimized  {final_refined_prompt}
"""
generated_code_tot = generate_code(prompt_tot)
print("Tree of thought ouput :")
print(generated_code_tot)


Tree of thought ouput :
# def fibonacci_recursive(n):
 
if n = 1:
 return recursion return fi bonacchi_ - a + fib % b return (return 0) * (n // 2) + (fibon__-rec) / 2 return 1 + fet * 2 + 1 * c * 3 + 2 * 1 & d * 4 + 3 * 5 + 4 * 6 + 5 * 7 + 6 * 8 + 8 * 9 + 7 * 10 + 9 * 12 + 10 * 13 + 12 * 14 + 13 * 15 + 15 * 16 + 16 * 18 + 18 * 20 + 20 * 24 + 24 * 26 + 23 * 25 + 25 * 30 + 30 * 40 + 35 * 60 + 90 * 100 + 100 * 150 + 150 * 200 + 300 * 300 + 200 * 400 + 400 * 500 + 600 * 1000 + 1000 * 600 + 800 * 800 + 1600 * 1600 + 1800 * 1800 + 1900 * 2000 + 2000 * 1500 + 1500 * 700 + 1920 * 1900 + 19200 * 2001 + 2002 * 2003 + 2003 * 2005 +


In [109]:
# for TOT_Fixed. as the max_length parameter had to be changed
def generate_code_tot_fixed(prompt):

    tokenizer.pad_token = tokenizer.eos_token


    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)


    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=500 , #500 , 300 changed this for more structured prompt ouput. also for TOT prompt
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.3,  #changed to from 0.3 to 0.5 for tot
        top_p=0.8,  #changed from 0.5 to 0.8 for tot
        top_k=40,  # changed from 50 to 40
        pad_token_id=tokenizer.eos_token_id,  # Ensure padding token is set
    )


    generated_code_tot_fixed = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code_tot_fixed




In [110]:
prompt_tot_fixed = """
# Let's explore multiple ways to implement the Fibonacci series.
# Approach 1: Using recursion.
def fibonacci_recursive(n):
    if n <= 1:
        return n
    return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

# Approach 2: Using iteration.
def fibonacci_iterative(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

# Approach 3: Using memoization for efficiency.
def fibonacci_memoization(n, memo={}):
    if n in memo:
        return memo[n]
    if n <= 1:
        return n
    memo[n] = fibonacci_memoization(n - 1, memo) + fibonacci_memoization(n - 2, memo)
    return memo[n]

# Approach 4: Optimized Fibonacci function using only Python (NO external imports allowed).
# Step 1: Define an efficient function that computes Fibonacci numbers.
# Step 2: Use dynamic programming or an iterative approach.
# Step 3: Return the Fibonacci number at position n.

Now, generate a function: {refined_prompt}
"""

generated_code_tot_fixed_answer = generate_code_tot_fixed(prompt_tot_fixed)
print("Tree-of-Thought Output (Fixed):\n", generated_code_tot_fixed_answer)


Tree-of-Thought Output (Fixed):
 n = int(input()) a = 0 b = 1 while i  m: if j == k: break c = (a + d) // 2 elif (c % 2) = =0: print(c // 2) else: for x in range(1, 10): while (x != 1): mid = mid + 1 y = math.ceil(x) print (y // 3)


In [111]:
def generate_multiple_codes(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        generated_code = generate_code(prompt)  # Call the existing function
        outputs.append(generated_code)

    # Choose the most frequent response
    most_common_output = max(set(outputs), key=outputs.count)
    return most_common_output


best_output = generate_multiple_codes(final_refined_prompt)
print("Self-Consistency Output:\n", best_output)


Self-Consistency Output:
 n = int(input()) 
if 0  stdout.write('n'):
 print(n) else:
 x = ''.join([s 
for _ in range(s)]) print()


In [113]:
prompt_instruction_based = """
# Instruction:
# 1. Define a function named gcd that takes two integers as input.
# 2. Use the Euclidean algorithm to compute the greatest common divisor.
# 3. Ensure that the function returns the correct output.

Now, generate a function: {final_refined_prompt}
"""

generated_code_instruction = generate_code(prompt_instruction_based)
print("Instruction-Based Output:\n", generated_code_instruction)


Instruction-Based Output:
 gcd = [] 
for i in range(2,int(input())+1):
 
if n % m == 0:
 continue el
if 2 * j = = -1:
 break a = 2*j b = 1 c = (a*b)//2 dp = list(map(lambda x:
 int, y:
 (x*y))%2) f = inf 
for item in s:
 
for element in item:
 print(element)


### Evaluation Metrics
### Trying to implement
* BLEU
* ROGUE
* CodeBLEU

### Problem in uploading the CodeBLEU repository


In [85]:
!git clone https://github.com/microsoft/CodeXGLUE.git
!git clone https://github.com/microsoft/CodeXGLUE.git
!pip install -r CodeXGLUE/Code-CodeBLEU/requirements.txt

!pip freeze > requirements.txt

Cloning into 'CodeXGLUE'...
remote: Enumerating objects: 3373, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 3373 (delta 218), reused 211 (delta 211), pack-reused 3124 (from 3)
Receiving objects: 100% (3373/3373), 213.15 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (1740/1740), done.
Updating files: 100% (400/400), done.
fatal: destination path 'CodeXGLUE' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'CodeXGLUE/Code-CodeBLEU/requirements.txt'


### Metrics for use case of "Calculating factorial"

### Expreimented with other use cases as well.
### The models are taking too long to fine tune and generate relevant code.
### This affected the execution and analysis of metrics

In [142]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Define expected (reference) outputs
expected_output_prime = """
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
"""

expected_output_factorial = """
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n-1)
"""

# Replace these with the actual generated outputs stored in variables
generated_outputs = {
    "Few-Shot 1": generated_code_FS1,
    "Few-Shot 2": generated_code_FS2,
    "Chain-of-Thought": generated_code_cot,
    "Zero-Shot": prompt_zero_shot,
    "Tree-of-Thought": generated_code_tot,
    "Instruction-Based": generated_code_instruction
}

expected_outputs = {
    "Few-Shot 1": expected_output_factorial,
    "Few-Shot 2": expected_output_factorial,
    "Zero-Shot": expected_output_factorial,
    "Chain-of-Thought": expected_output_factorial,
    "Tree-of-Thought": expected_output_factorial,
    "Instruction-Based": expected_output_prime
}

# BLEU Score Calculation
def calculate_bleu(reference, generated):
    reference = [reference.split()]  # BLEU requires a list of tokenized words
    generated = generated.split()    # Tokenizing the generated output
    smoothing_function = SmoothingFunction().method4
    if not generated:
        return 0.0
    return sentence_bleu(reference, generated)

# ROUGE Score Calculation
def calculate_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores


evaluation_results = {}
# Evaluate each generated output
for key in generated_outputs.keys():
    generated_code = generated_outputs[key]
    expected_code = expected_outputs[key]

    bleu_score = calculate_bleu(expected_code, generated_code)
    rouge_scores = calculate_rouge(expected_code, generated_code)

    evaluation_results[key] = {
        "BLEU": bleu_score,
        "ROUGE-1": rouge_scores['rouge1'].fmeasure,
        "ROUGE-2": rouge_scores['rouge2'].fmeasure,
        "ROUGE-L": rouge_scores['rougeL'].fmeasure

    }

import pandas as pd
df_results = pd.DataFrame(evaluation_results).T  # Storing the scores in   DataFrame for better presentation
print(df_results)



                            BLEU   ROUGE-1   ROUGE-2   ROUGE-L
Few-Shot 1         8.726095e-232  0.217391  0.000000  0.217391
Few-Shot 2         1.024491e-231  0.285714  0.000000  0.238095
Chain-of-Thought   9.641193e-232  0.280000  0.083333  0.200000
Zero-Shot          8.460552e-232  0.148148  0.000000  0.148148
Tree-of-Thought    2.349320e-155  0.165289  0.084034  0.148760
Instruction-Based   2.000841e-78  0.307692  0.184211  0.256410


In [136]:
# Evaluate each generated output
print("\n🔹 **Evaluation Results:**")
print("| Model                | BLEU Score                  | ROUGE-1 F                   | ROUGE-2 F                   | ROUGE-L F |")
print("|----------------------|-----------------------------|-----------------------------|-----------------------------|-----------------------------|")
for key in generated_outputs.keys():
    generated_code = generated_outputs[key]
    expected_code = expected_outputs[key]

    bleu_score = calculate_bleu(expected_code, generated_code)
    rouge_scores = calculate_rouge(expected_code, generated_code)

    print(f"| {key:<20} | {bleu_score:<10} | {rouge_scores['rouge1'].fmeasure:<10} | {rouge_scores['rouge2'].fmeasure:<10} | {rouge_scores['rougeL'].fmeasure:<10} |")



🔹 **Evaluation Results:**
| Model                | BLEU Score                  | ROUGE-1 F                   | ROUGE-2 F                   | ROUGE-L F |
|----------------------|-----------------------------|-----------------------------|-----------------------------|-----------------------------|
| Few-Shot 1           | 8.726094729337945e-232 | 0.21739130434782608 | 0.0        | 0.21739130434782608 |
| Few-Shot 2           | 1.0244914152188952e-231 | 0.28571428571428575 | 0.0        | 0.23809523809523808 |
| Chain-of-Thought     | 9.641193013181824e-232 | 0.28       | 0.08333333333333333 | 0.2        |
| Zero-Shot            | 8.460552185460498e-232 | 0.14814814814814817 | 0.0        | 0.14814814814814817 |
| Tree-of-Thought      | 2.349319979074875e-155 | 0.1652892561983471 | 0.08403361344537814 | 0.14876033057851237 |
| Instruction-Based    | 2.0008414238016498e-78 | 0.30769230769230765 | 0.18421052631578946 | 0.2564102564102564 |


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
# Example: Save your data (e.g., a file) to Google Drive
with open('/content/drive/My Drive/my_data.txt', 'w') as f:
    f.write('This is my important data.')


### Integrating the UI using Streamlit

The app.py file contains the code for your Streamlit application. When running this code in Google Colab, it starts a local development server within the Colab environment. To make this local server accessible over the internet, ngrok is used.

Ngrok is a tool that creates a secure tunnel from a public URL on the local server running in Google Colab. This allows the sharing local development server with others, enabling the access of the remote application from anywhere

### Installing the libraries

In [149]:
!pip install pyngrok
!pip install streamlit transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 788.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

#### Running the Streamlit app.

#### The authentication token was taken from the Streamlit portal
#### https://dashboard.ngrok.com/get-started/your-authtoken


### Streamlit app is run and is made accessible over the internet via Ngrok, a tool that exposes local servers to the web.

In [150]:
%%writefile app.py

import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer (Replace with your trained model path or Hugging Face model)
MODEL_NAME = "codeparrot/codeparrot-small"  # Change if using a different model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Streamlit UI
st.title("💬 Code Generation Chatbot")

# User input
user_input = st.text_area("Enter your qyery:", "")

if st.button("Generate Code"):
    if user_input.strip():
        # Tokenize input
        input_ids = tokenizer(user_input, return_tensors="pt").input_ids

        # Generate response
        output_ids = model.generate(input_ids, max_length=150)
        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Display output
        st.subheader("Response:")
        st.write(response)
    else:
        st.warning("Please enter a prompt before generating.")



Overwriting app.py


In [152]:
from pyngrok import ngrok
import subprocess
import time

#  ngrok authtoken required for providing accesibility to the web
ngrok.set_auth_token('2m6abmNJKj2N8DFTiAm0PhWSzQB_3AXNba277XQFygLDLYpGC')

# Running app.py
subprocess.Popen(["streamlit", "run", "app.py"])

# since the app is run multiple times, this will pause the program for 10 seconds, giving time before restarting the chatbot.
time.sleep(10)

# setting the tunnel
public_url = ngrok.connect(8501, "http")
print('ChatBOt is now active. Click on the link')
print(f"Streamlit app is live at {public_url}")  # this public url


ChatBOt is now active. Click on the link
Streamlit app is live at NgrokTunnel: "https://9d12-34-125-56-24.ngrok-free.app" -> "http://localhost:8501"


In [146]:
from pyngrok import ngrok

# Kill existing tunnels if necessary
ngrok.kill()


### Clicking on the above live app link, opens the chatbot